In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/12500/1375107/compressed/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1620923137&Signature=BtYWG6%2BLbAiwYeplmX8a31mzKZkLmmQWOeRwPKUUE%2FURgcpy7Efy64AVjtVqWARTwx%2Ft2CENCA8np0IBx%2BpDGIXM3G4gGdv%2FIlEe4m%2Bebjl4o6K2OltbfbhbzS%2BP5ZOLsDvoQmj3XZG%2FmBnHCVfvH4HU5wskRM9ZdB5M47A6I%2BNietXdAxgqmh04peA21jZR%2Freayp2EQRo5g4gVLgsvXegHmTJ%2Fr%2B1St7q0KCC4KUDfwYNrxrYQUmZmz1o78CgWsdJf9PTQgbBedE79MbHuFp3r%2FeyQpGwH5vROS0RP0fofFYkuthtv2NLYI4ri62pYeGhy9YxiAfSfAcRQBvCozg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip" -c -O 'train.csv.zip'
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/12500/1375107/compressed/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1620923168&Signature=L95ETvXXHp0mPtpxqrNAjf5RHQi0%2Bdaf2nB3vHnW2LHfAOfb9MT1OkSXKO3XApY28s52yD0jXAhMcP7Dl5n75XigbJOpYmJjsEUlm9QEd56JaZ4SJ3%2F%2Bd4DCb1cFQcpJnCz3DKP6C5p5BiAVGuBs8G%2Bupr6j0DEZH60gLwOeXYp6M7HOS2KGrdyPX11R8zx95PTnoJZeLE8xlYRDdbOkzs3XWo6bAYYy0Ejx13l%2BblwMXPiNm4OPfiQ3U%2Bu4IyyiismjFHLrANFNNNyGfR70uUclARt4CUHYiqZSkg9hNqGIM%2FPt60ptWLuc2MyhHzvp5oK5U7HTsBQ87U8k0oPhcQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.csv.zip" -c -O 'test.csv.zip'
!pip install sentencepiece
!cp /content/drive/MyDrive/Colab\ Notebooks/AAIC/Case\ Study\ 1/tokenization.py /content/

--2021-05-13 11:19:38--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/12500/1375107/compressed/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1620923137&Signature=BtYWG6%2BLbAiwYeplmX8a31mzKZkLmmQWOeRwPKUUE%2FURgcpy7Efy64AVjtVqWARTwx%2Ft2CENCA8np0IBx%2BpDGIXM3G4gGdv%2FIlEe4m%2Bebjl4o6K2OltbfbhbzS%2BP5ZOLsDvoQmj3XZG%2FmBnHCVfvH4HU5wskRM9ZdB5M47A6I%2BNietXdAxgqmh04peA21jZR%2Freayp2EQRo5g4gVLgsvXegHmTJ%2Fr%2B1St7q0KCC4KUDfwYNrxrYQUmZmz1o78CgWsdJf9PTQgbBedE79MbHuFp3r%2FeyQpGwH5vROS0RP0fofFYkuthtv2NLYI4ri62pYeGhy9YxiAfSfAcRQBvCozg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.96.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289820185 (276M) [application/zip]
Saving to: ‘train.csv.zip’

train.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from tqdm import tqdm
import pickle

import sys
import tokenization

##Source: https://stackoverflow.com/a/14463362/8211103
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('/content/train.csv.zip')

In [ ]:
contraction_mapping = {
    "Trump's" : 'trump is',"'cause": 'because',',cause': 'because',';cause': 'because',"ain't": 'am not','ain,t': 'am not',
    'ain;t': 'am not','ain´t': 'am not','ain’t': 'am not',"aren't": 'are not',
    'aren,t': 'are not','aren;t': 'are not','aren´t': 'are not','aren’t': 'are not',"can't": 'cannot',"can't've": 'cannot have','can,t': 'cannot','can,t,ve': 'cannot have',
    'can;t': 'cannot','can;t;ve': 'cannot have',
    'can´t': 'cannot','can´t´ve': 'cannot have','can’t': 'cannot','can’t’ve': 'cannot have',
    "could've": 'could have','could,ve': 'could have','could;ve': 'could have',"couldn't": 'could not',"couldn't've": 'could not have','couldn,t': 'could not','couldn,t,ve': 'could not have','couldn;t': 'could not',
    'couldn;t;ve': 'could not have','couldn´t': 'could not',
    'couldn´t´ve': 'could not have','couldn’t': 'could not','couldn’t’ve': 'could not have','could´ve': 'could have',
    'could’ve': 'could have',"didn't": 'did not','didn,t': 'did not','didn;t': 'did not','didn´t': 'did not',
    'didn’t': 'did not',"doesn't": 'does not','doesn,t': 'does not','doesn;t': 'does not','doesn´t': 'does not',
    'doesn’t': 'does not',"don't": 'do not','don,t': 'do not','don;t': 'do not','don´t': 'do not','don’t': 'do not',
    "hadn't": 'had not',"hadn't've": 'had not have','hadn,t': 'had not','hadn,t,ve': 'had not have','hadn;t': 'had not',
    'hadn;t;ve': 'had not have','hadn´t': 'had not','hadn´t´ve': 'had not have','hadn’t': 'had not','hadn’t’ve': 'had not have',"hasn't": 'has not','hasn,t': 'has not','hasn;t': 'has not','hasn´t': 'has not','hasn’t': 'has not',
    "haven't": 'have not','haven,t': 'have not','haven;t': 'have not','haven´t': 'have not','haven’t': 'have not',"he'd": 'he would',
    "he'd've": 'he would have',"he'll": 'he will',
    "he's": 'he is','he,d': 'he would','he,d,ve': 'he would have','he,ll': 'he will','he,s': 'he is','he;d': 'he would',
    'he;d;ve': 'he would have','he;ll': 'he will','he;s': 'he is','he´d': 'he would','he´d´ve': 'he would have','he´ll': 'he will',
    'he´s': 'he is','he’d': 'he would','he’d’ve': 'he would have','he’ll': 'he will','he’s': 'he is',"how'd": 'how did',"how'll": 'how will',
    "how's": 'how is','how,d': 'how did','how,ll': 'how will','how,s': 'how is','how;d': 'how did','how;ll': 'how will',
    'how;s': 'how is','how´d': 'how did','how´ll': 'how will','how´s': 'how is','how’d': 'how did','how’ll': 'how will',
    'how’s': 'how is',"i'd": 'i would',"i'll": 'i will',"i'm": 'i am',"i've": 'i have','i,d': 'i would','i,ll': 'i will',
    'i,m': 'i am','i,ve': 'i have','i;d': 'i would','i;ll': 'i will','i;m': 'i am','i;ve': 'i have',"isn't": 'is not',
    'isn,t': 'is not','isn;t': 'is not','isn´t': 'is not','isn’t': 'is not',"it'd": 'it would',"it'll": 'it will',"It's":'it is',
    "it's": 'it is','it,d': 'it would','it,ll': 'it will','it,s': 'it is','it;d': 'it would','it;ll': 'it will','it;s': 'it is','it´d': 'it would','it´ll': 'it will','it´s': 'it is',
    'it’d': 'it would','it’ll': 'it will','it’s': 'it is',
    'i´d': 'i would','i´ll': 'i will','i´m': 'i am','i´ve': 'i have','i’d': 'i would','i’ll': 'i will','i’m': 'i am',
    'i’ve': 'i have',"let's": 'let us','let,s': 'let us','let;s': 'let us','let´s': 'let us',
    'let’s': 'let us',"ma'am": 'madam','ma,am': 'madam','ma;am': 'madam',"mayn't": 'may not','mayn,t': 'may not','mayn;t': 'may not',
    'mayn´t': 'may not','mayn’t': 'may not','ma´am': 'madam','ma’am': 'madam',"might've": 'might have','might,ve': 'might have','might;ve': 'might have',"mightn't": 'might not','mightn,t': 'might not','mightn;t': 'might not','mightn´t': 'might not',
    'mightn’t': 'might not','might´ve': 'might have','might’ve': 'might have',"must've": 'must have','must,ve': 'must have','must;ve': 'must have',
    "mustn't": 'must not','mustn,t': 'must not','mustn;t': 'must not','mustn´t': 'must not','mustn’t': 'must not','must´ve': 'must have',
    'must’ve': 'must have',"needn't": 'need not','needn,t': 'need not','needn;t': 'need not','needn´t': 'need not','needn’t': 'need not',"oughtn't": 'ought not','oughtn,t': 'ought not','oughtn;t': 'ought not',
    'oughtn´t': 'ought not','oughtn’t': 'ought not',"sha'n't": 'shall not','sha,n,t': 'shall not','sha;n;t': 'shall not',"shan't": 'shall not',
    'shan,t': 'shall not','shan;t': 'shall not','shan´t': 'shall not','shan’t': 'shall not','sha´n´t': 'shall not','sha’n’t': 'shall not',
    "she'd": 'she would',"she'll": 'she will',"she's": 'she is','she,d': 'she would','she,ll': 'she will',
    'she,s': 'she is','she;d': 'she would','she;ll': 'she will','she;s': 'she is','she´d': 'she would','she´ll': 'she will',
    'she´s': 'she is','she’d': 'she would','she’ll': 'she will','she’s': 'she is',"should've": 'should have','should,ve': 'should have','should;ve': 'should have',
    "shouldn't": 'should not','shouldn,t': 'should not','shouldn;t': 'should not','shouldn´t': 'should not','shouldn’t': 'should not','should´ve': 'should have',
    'should’ve': 'should have',"that'd": 'that would',"that's": 'that is','that,d': 'that would','that,s': 'that is','that;d': 'that would',
    'that;s': 'that is','that´d': 'that would','that´s': 'that is','that’d': 'that would','that’s': 'that is',"there'd": 'there had',
    "there's": 'there is','there,d': 'there had','there,s': 'there is','there;d': 'there had','there;s': 'there is',
    'there´d': 'there had','there´s': 'there is','there’d': 'there had','there’s': 'there is',
    "they'd": 'they would',"they'll": 'they will',"they're": 'they are',"they've": 'they have',
    'they,d': 'they would','they,ll': 'they will','they,re': 'they are','they,ve': 'they have','they;d': 'they would','they;ll': 'they will','they;re': 'they are',
    'they;ve': 'they have','they´d': 'they would','they´ll': 'they will','they´re': 'they are','they´ve': 'they have','they’d': 'they would','they’ll': 'they will',
    'they’re': 'they are','they’ve': 'they have',"wasn't": 'was not','wasn,t': 'was not','wasn;t': 'was not','wasn´t': 'was not',
    'wasn’t': 'was not',"we'd": 'we would',"we'll": 'we will',"we're": 'we are',"we've": 'we have','we,d': 'we would','we,ll': 'we will',
    'we,re': 'we are','we,ve': 'we have','we;d': 'we would','we;ll': 'we will','we;re': 'we are','we;ve': 'we have',
    "weren't": 'were not','weren,t': 'were not','weren;t': 'were not','weren´t': 'were not','weren’t': 'were not','we´d': 'we would','we´ll': 'we will',
    'we´re': 'we are','we´ve': 'we have','we’d': 'we would','we’ll': 'we will','we’re': 'we are','we’ve': 'we have',"what'll": 'what will',"what're": 'what are',"what's": 'what is',
    "what've": 'what have','what,ll': 'what will','what,re': 'what are','what,s': 'what is','what,ve': 'what have','what;ll': 'what will','what;re': 'what are',
    'what;s': 'what is','what;ve': 'what have','what´ll': 'what will',
    'what´re': 'what are','what´s': 'what is','what´ve': 'what have','what’ll': 'what will','what’re': 'what are','what’s': 'what is',
    'what’ve': 'what have',"where'd": 'where did',"where's": 'where is','where,d': 'where did','where,s': 'where is','where;d': 'where did',
    'where;s': 'where is','where´d': 'where did','where´s': 'where is','where’d': 'where did','where’s': 'where is',
    "who'll": 'who will',"who's": 'who is','who,ll': 'who will','who,s': 'who is','who;ll': 'who will','who;s': 'who is',
    'who´ll': 'who will','who´s': 'who is','who’ll': 'who will','who’s': 'who is',"won't": 'will not','won,t': 'will not','won;t': 'will not',
    'won´t': 'will not','won’t': 'will not',"wouldn't": 'would not','wouldn,t': 'would not','wouldn;t': 'would not','wouldn´t': 'would not',
    'wouldn’t': 'would not',"you'd": 'you would',"you'll": 'you will',"you're": 'you are','you,d': 'you would','you,ll': 'you will',
    'you,re': 'you are','you;d': 'you would','you;ll': 'you will',
    'you;re': 'you are','you´d': 'you would','you´ll': 'you will','you´re': 'you are','you’d': 'you would','you’ll': 'you will','you’re': 'you are',
    '´cause': 'because','’cause': 'because',"you've": "you have","could'nt": 'could not',
    "havn't": 'have not',"here’s": "here is",'i""m': 'i am',"i'am": 'i am',"i'l": "i will","i'v": 'i have',"wan't": 'want',"was'nt": "was not","who'd": "who would",
    "who're": "who are","who've": "who have","why'd": "why would","would've": "would have","y'all": "you all","y'know": "you know","you.i": "you i",
    "your'e": "you are","arn't": "are not","agains't": "against","c'mon": "common","doens't": "does not",'don""t': "do not","dosen't": "does not",
    "dosn't": "does not","shoudn't": "should not","that'll": "that will","there'll": "there will","there're": "there are",
    "this'll": "this all","u're": "you are", "ya'll": "you all","you'r": "you are","you’ve": "you have","d'int": "did not","did'nt": "did not","din't": "did not","dont't": "do not","gov't": "government",
    "i'ma": "i am","is'nt": "is not","‘I":'I',
    'ᴀɴᴅ':'and','ᴛʜᴇ':'the','ʜᴏᴍᴇ':'home','ᴜᴘ':'up','ʙʏ':'by','ᴀᴛ':'at','…and':'and','civilbeat':'civil beat',\
    'TrumpCare':'Trump care','Trumpcare':'Trump care', 'OBAMAcare':'Obama care','ᴄʜᴇᴄᴋ':'check','ғᴏʀ':'for','ᴛʜɪs':'this','ᴄᴏᴍᴘᴜᴛᴇʀ':'computer',\
    'ᴍᴏɴᴛʜ':'month','ᴡᴏʀᴋɪɴɢ':'working','ᴊᴏʙ':'job','ғʀᴏᴍ':'from','Sᴛᴀʀᴛ':'start','gubmit':'submit','CO₂':'carbon dioxide','ғɪʀsᴛ':'first',\
    'ᴇɴᴅ':'end','ᴄᴀɴ':'can','ʜᴀᴠᴇ':'have','ᴛᴏ':'to','ʟɪɴᴋ':'link','ᴏғ':'of','ʜᴏᴜʀʟʏ':'hourly','ᴡᴇᴇᴋ':'week','ᴇɴᴅ':'end','ᴇxᴛʀᴀ':'extra',\
    'Gʀᴇᴀᴛ':'great','sᴛᴜᴅᴇɴᴛs':'student','sᴛᴀʏ':'stay','ᴍᴏᴍs':'mother','ᴏʀ':'or','ᴀɴʏᴏɴᴇ':'anyone','ɴᴇᴇᴅɪɴɢ':'needing','ᴀɴ':'an','ɪɴᴄᴏᴍᴇ':'income',\
    'ʀᴇʟɪᴀʙʟᴇ':'reliable','ғɪʀsᴛ':'first','ʏᴏᴜʀ':'your','sɪɢɴɪɴɢ':'signing','ʙᴏᴛᴛᴏᴍ':'bottom','ғᴏʟʟᴏᴡɪɴɢ':'following','Mᴀᴋᴇ':'make',\
    'ᴄᴏɴɴᴇᴄᴛɪᴏɴ':'connection','ɪɴᴛᴇʀɴᴇᴛ':'internet','financialpost':'financial post', 'ʜaᴠᴇ':' have ', 'ᴄaɴ':' can ', 'Maᴋᴇ':' make ', 'ʀᴇʟɪaʙʟᴇ':' reliable ', 'ɴᴇᴇᴅ':' need ',
    'ᴏɴʟʏ':' only ', 'ᴇxᴛʀa':' extra ', 'aɴ':' an ', 'aɴʏᴏɴᴇ':' anyone ', 'sᴛaʏ':' stay ', 'Sᴛaʀᴛ':' start', 'SHOPO':'shop',
    }

def contractions(txt):
    for k,v in contraction_mapping.items():
        txt = txt.replace(k,v)
    return txt


def preprocessing(txt):
    txt = contractions(txt)
    txt = re.sub("[^a-zA-Z]+", " ", txt)
    txt = txt.lower()
    txt = re.sub(' +', ' ',txt)
    return txt

In [ ]:
data['comment_text'] = data['comment_text'].apply(preprocessing)

In [ ]:
data = data[['comment_text','target']]
data['target'] = (data['target']>=.5).astype('int')

y = data['target'] 
x_train,x_test,y_train,y_test = train_test_split(data['comment_text'],y,stratify=y,test_size=0.2)

x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

del data

In [ ]:
lenar = [len(line.split()) for line in x_train]
print(f"25th percentile value is {np.percentile(lenar,25)}")
print(f"50th percentile value is {np.percentile(lenar,50)}")
print(f"75th percentile value is {np.percentile(lenar,75,)}")
print(f"90th percentile value is {np.percentile(lenar,90)}")
print(f"95th percentile value is {np.percentile(lenar,95)}")
print(f"99th percentile value is {np.percentile(lenar,99)}")
print(f"100th percentile value is {np.percentile(lenar,100)}")

25th percentile value is 17.0
50th percentile value is 36.0
75th percentile value is 73.0
90th percentile value is 130.0
95th percentile value is 161.0
99th percentile value is 180.0
100th percentile value is 320.0


In [ ]:
tf.keras.backend.clear_session()
max_seq_length = 180
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
bert_model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=pooled_output)
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 180)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 180)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 180)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def apply_tokenizer(x_train):
    X_train_tokens = []
    X_train_mask = []
    X_train_segment = []
    for txt in tqdm(x_train):
        tok = tokenizer.tokenize(txt)
        if len(tok) > max_seq_length-2:
            tok = tok[0:(max_seq_length-2)]
        tok = ['[CLS]',*tok,'[SEP]']
        X_train_mask.append(np.array([1]*len(tok)+[0]*(max_seq_length-len(tok))))
        if len(tok)< max_seq_length:
            tok += (max_seq_length - len(tok)) * ['[PAD]']
        tok = np.array(tokenizer.convert_tokens_to_ids(tok))
        X_train_tokens.append(tok)
        X_train_segment.append(np.array([0]*max_seq_length))
    return np.array(X_train_tokens),np.array(X_train_mask),np.array(X_train_segment)

In [ ]:
x_test_tokens,x_test_mask,x_test_segment = apply_tokenizer(x_test)
del x_test
print(x_test_tokens.shape,x_test_mask.shape,x_test_segment.shape)
pickle.dump((x_test_tokens, x_test_mask, x_test_segment, y_test),open('x_test_data.pkl','wb'))
x_test_pooled_output=bert_model.predict([x_test_tokens,x_test_mask,x_test_segment])
del x_test_tokens
del x_test_mask
del x_test_segment
pickle.dump((x_test_pooled_output,y_test),open('X_TEST_FE_BERT.pkl','wb'))
del x_test_pooled_output
del y_test

100%|██████████| 360975/360975 [06:12<00:00, 970.02it/s]


(360975, 180) (360975, 180) (360975, 180)


In [ ]:
x_train_tokens,x_train_mask,x_train_segment = apply_tokenizer(x_train)
del x_train
print(x_train_tokens.shape,x_train_mask.shape,x_train_segment.shape)
pickle.dump((x_train_tokens, x_train_mask, x_train_segment, y_train),open('x_train_data.pkl','wb'))
x_train_pooled_output=bert_model.predict([x_train_tokens,x_train_mask,x_train_segment])
del x_train_tokens
del x_train_mask
del x_train_segment
pickle.dump((x_train_pooled_output,y_train),open('X_TRAIN_FE_BERT.pkl','wb'), protocol=4)
del x_train_pooled_output
del y_train

In [ ]:
!cp /content/X_TEST_FE_BERT.pkl /content/drive/MyDrive/Colab\ Notebooks/AAIC/Case\ Study\ 1/X_TEST_FE_BERT.pkl
!cp /content/X_TRAIN_FE_BERT.pkl /content/drive/MyDrive/Colab\ Notebooks/AAIC/Case\ Study\ 1/X_TRAIN_FE_BERT.pkl

In [ ]:
test = pd.read_csv('/content/test.csv.zip')
test['comment_text'] = test['comment_text'].apply(preprocessing)

In [ ]:
test = test['comment_text']

In [ ]:
test_tokens,test_mask,test_segment = apply_tokenizer(test)
del test
print(test_tokens.shape,test_mask.shape,test_segment.shape)
test_pooled_output=bert_model.predict([test_tokens,test_mask,test_segment])
del test_tokens
del test_mask
del test_segment
pickle.dump((test_pooled_output),open('TEST_FE_BERT.pkl','wb'))
del test_pooled_output

100%|██████████| 97320/97320 [01:24<00:00, 1155.53it/s]


(97320, 180) (97320, 180) (97320, 180)


In [ ]:
!cp /content/TEST_FE_BERT.pkl /content/drive/MyDrive/Colab\ Notebooks/AAIC/Case\ Study\ 1/TEST_FE_BERT.pkl